# Linear MDPはクリップなしで解けると思う

[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb)では価値をクリップする方法で解きました．
個人的にクリップせずに解ける気がしたので，メモしておきます．

表記：[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb)と同じ

## モデルの推定について

### Clipの復習

$\mu_h^{\star} \phi\left(s_h^i, a_h^i\right)=P_h\left(\cdot \mid s_h^i, a_h^i\right)$なので，次のリッジ回帰をすれば$\mu_h^\star$が普遍推定できます：
$$
\widehat{\mu}_h^n=\operatorname{argmin}_{\mu \in \mathbb{R}^{|\mathcal{S}| \times d}} \sum_{i=0}^{n-1}\left\|\mu \phi\left(s_h^i, a_h^i\right)-\delta\left(s_{h+1}^i\right)\right\|_2^2+\lambda\|\mu\|_F^2 .
$$
この解析解は
$$
\widehat{\mu}_h^n=\sum_{i=0}^{n-1} \delta\left(s_{h+1}^i\right) \phi\left(s_h^i, a_h^i\right)^{\top}\left(\Lambda_h^n\right)^{-1}
$$
でした．
Clipするアプローチでは，

$$
\widehat{P}_h^n(\cdot \mid s, a) \cdot V:=\left(\widehat{\mu}_h^n \phi(s, a)\right) \cdot V=\phi(s, a)^{\top} \sum_{i=0}^{n-1}\left(\Lambda_h^n\right)^{-1} \phi\left(s_h^i, a_h^i\right) V\left(s_{h+1}^i\right)
$$

の形を考えていました．

### Clipなし

さて，$\widehat{\mu}_h^n(s')\phi(s, a) = \widehat{P}(s'\mid s, a)$としてきました．各$s, a$で，この$\widehat{P}(\cdot\mid s, a)$を$\Delta(S)$に射影してみましょう．
つまり，　
$$
\widetilde{P}(\cdot \mid s, a) = \operatorname{Proj}_{\Delta(S)}(\widehat{P}(\cdot\mid s, a))
$$
とします．これは計算がむずそうに見えますが，実は簡単です．

---

各$s' \in S$について，
$$
\widehat{\mu}_h^n(s') \in \mathbb{R}^d=\sum_{i=0}^{n-1} \mathbb{I}[s' = s_{h+1}^i] \underbrace{\phi\left(s_h^i, a_h^i\right)^{\top}\left(\Lambda_h^n\right)^{-1}}_{\in \mathbb{R}^d}
$$
および

$$
\widehat{P}_h^n(s' \mid s, a) = 
\widehat{\mu}_h^n(s')\phi(s, a)= \left(\sum_{i=0}^{n-1} \mathbb{I}[s' = s_{h+1}^i]\phi\left(s_h^i, a_h^i\right)^{\top}\left(\Lambda_h^n\right)^{-1}\right)\phi(s, a) \in \R
$$

なので，$\widehat{P}(\cdot \mid s, a)$のうち，$0$以外の値になるのは最大でも$n$個しかありません．

ここで，$\operatorname{Proj}_{\Delta(S)}\widehat{P}(\cdot \mid s, a)$を[Simplexへの射影アルゴリズム](https://arxiv.org/pdf/1309.1541)を使って実現することを考えましょう．射影アルゴリズムは次の通りです：

* 入力： $d$次元ベクトル：$p \in \mathbb{R}^d$
* $p$をソートして，出てきたベクトルを$u$とする：$u_1 \geq u_2 \geq \cdots \geq u_d$
* $\rho=\max \left\{1 \leq j \leq D: u_j+\frac{1}{j}\left(1-\sum_{i=1}^j u_i\right)>0\right\}$とする．
* $\lambda=\frac{1}{\rho}\left(1-\sum_{i=1}^\rho u_i\right)$
* 出力 $x$： $x_i=\max \left\{y_i+\lambda, 0\right\}, i=1, \ldots, D$

**Linear MDPでの射影アルゴリズム**

これを今回の場合に拡張しましょう．最終的には$\Delta(S)$への射影を出します：

<!-- * $\mathcal{S}_{\neq 0} = \{s' \in \mathcal{S}\mid \widehat{P}^n_h(s'\mid s, a) \neq 0\}$を$\widehat{P}^n_h$が0以外の値を取る状態とします． -->
<!-- * $\mathcal{P} = \left\{\widehat{P}^n_h(s_i\mid s, a) \mid s_i \in \mathcal{S}\right\}$とします． -->
<!-- $\widehat{P}^n_h(s_i \mid s, a)$のうち，非ゼロの要素は最大でも$n$個しかないので，$|\mathcal{P}| \leq \min\{n+1, S\}$です． -->
* $\mathcal{S}$内の状態に対して，
$\widehat{P}^n_h(s'_1 \mid s, a) \geq \widehat{P}^n_h(s'_2 \mid s, a)\geq \dots \geq \widehat{P}^n_h(s'_S \mid s, a)$
となるように番号をつけます．
  * 非ゼロの要素は最大でも$n$個しかないので，このソートは長さ$\min \{n+1, S\}$の系列についての計算効率で達成できます．
  * $\widehat{P}^n_h$が0を取る状態数を$N_0$と置きましょう．
* $j=0$として，
  * $j' = 
  \begin{cases} 
  &j + N_0 \quad \text{ if } \widehat{P}^n_h(s_{j+1}'\mid s, a) = 0\\
  &j + 1 \quad \text{ otherwise }
  \end{cases} 
  $
  * $\widehat{P}^n_h(s_{j'}'\mid s, a)+\frac{1}{j'}\left(1-\sum_{i=1}^{j'} \widehat{P}^n_h(s_i\mid s, a)\right)\leq 0$もしくは$j' > S$ ならbreak
  * そうでないならば$j \leftarrow j'$
* $\lambda=\frac{1}{j}\left(1-\sum_{i=1}^j p_i\right)$
* 出力 $\widetilde{P}$： $\widetilde{P}(\cdot \mid s, a)=\max\left\{\widehat{P}^n_h(\cdot \mid s, a) + \lambda , 0\right\}$

この処理は上のprojectionと同じです．
実際，$\widehat{P}^n_h(s_{j+1}'\mid s, a) = 0$のとき，
$k \in j+1 \dots j + N_0$では
$$
\begin{aligned}
u_k + \frac{1}{k}\left(1-\sum_{i=1}^k u_i\right)
= 
\frac{1}{k}\left(1-\sum_{i=1}^j u_i\right)
\end{aligned}
$$
なので，符号が固定されることを利用して$0$になる部分をスキップしているだけです．

<!-- また，各$p_i$について，$n_i = |\{s'\in \mathcal{S} \mid \widehat{P}^n_h(s'\mid s, a) = p_i\}|$を$p_i$を取る状態の数とします． -->
<!-- これも非ゼロの値が$n$個しかないので，$n$個の状態をチェックすれば全部計算できます． -->
<!-- * $\widehat{\mathcal{S}}$の状態を$\widehat{s}_1, \dots, \widehat{s}_{\widehat{S}}$とします．
* $i=0...\widehat{S}$について，$\widehat{p}_i = \widehat{P}(\widehat{s}_i\mid s, a)$とおきます．
* $\widehat{p}_i$をソートして，出てきたベクトルを$u$とする．これは$\widehat{p}_i$についてソートすれば良いので，最大でも$n$個しか要素を考えなくて良いです．：$\widehat{u}_1 \geq \widehat{u}_2 \geq \cdots \geq \widehat{u}_{\widehat{S}+1}$ -->


## Clipなしのアルゴリズム

通常のアルゴリズムと同様の流れを考えますが，Clipしません．
次のボーナス関数を考えましょう：
$$
b_h^n(s, a)=\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}
$$

そして，次の価値反復法による更新を考えましょう：

$$
\begin{aligned}
& \widehat{V}_H^n(s)=0, \forall s \\
& \widehat{Q}_h^n(s, a)=
\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)} + 
\theta^{\star} \cdot \phi(s, a)+
\operatorname{Proj}_{\Delta(S)} (\phi(s, a)^{\top}\left(\widehat{\mu}_h^n\right)^{\top}) \widehat{V}_{h+1}^n \\
& \widehat{V}_h^n(s)=\max _a \widehat{Q}_h^n(s, a), \quad \pi_h^n(s)=\operatorname{argmax}_a \widehat{Q}_h^n(s, a)
\end{aligned}
$$


## リグレット解析


任意の$f \in \mathbb{R}^S$について，
$$
\begin{aligned}
& \left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot f\right| \\
& = \left|\left(\operatorname{Proj}_{\Delta(\mathcal{S})}[\widetilde{P}_h^n(\cdot \mid s, a)]-\operatorname{Proj}_{\Delta(\mathcal{S})}[P(\cdot \mid s, a)]\right) \cdot f\right| \\
& \leq \left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot f\right| 
\end{aligned}
$$

を示しましょう．

**これ成立しないから駄目ですね．$f=[1, 2]$，$P=[0.5, 0.5]$, $\hat{P}=[-0.5, 1]$とすると，$\widetilde{P}=[0, 1]$であり，明らかに成立しない**